In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.font_manager import FontProperties

# matplotlib.style.use('ggplot')
import matplotlib
# matplotlib.style.use('ggplot')
#matplotlib.style.use('default')
from numpy import *
import numpy as np
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 6)
import subprocess,shlex,sys,select,re,StringIO
def readHive(sql):
    text = ''
    err = ''
    cmd = shlex.split('hive -e '+ '"set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;'+sql+'"')
    print >>sys.stderr, cmd
    sys.stderr.flush()
    proc = subprocess.Popen(cmd, bufsize=0, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    while proc.poll() is None:
        ins, outs, errs = select.select([proc.stdout, proc.stderr], [], [])
        if proc.stdout in ins:
            text += proc.stdout.read()
        if proc.stderr in ins:
            c = proc.stderr.read(1)
            if c == '\n':
                print >>sys.stderr, err
                sys.stderr.flush()
                err = ''
            else:
                err += c
    text += proc.stdout.read()
    print >>sys.stderr, proc.stderr.read()
    sys.stderr.flush()
    header, content = text.split('\n',1)
    header = re.sub('\w*\.','', header)
    return pd.read_table(StringIO.StringIO(header+'\n'+content), error_bad_lines = False)

def f(x):
    x=str(x)
    tmp=x.split("//")
    if len(tmp)<=1:
        return x
    return '.'.join(x.strip().split('//')[1].split('/')[0].split('.')[-3:])

Populating the interactive namespace from numpy and matplotlib


In [75]:
imp={}
clk={}
ld={}

impclk={}
clkld={}

# imp[k712]=readHive("""select * from fraud_free.dimpv2 where year=2016 and month=7 and day=12""")
k712='712'
clk[k712]=readHive("""select * from fraud_free.dclkv2 where year=2016 and month=7 and day=12 and request_id is not null""")
ld[k712]=readHive("""select * from etl_output.landing where year=2016 and month=7 and day=12 and request_id is not null""")
k812='812'
clk[k812]=readHive("""select * from fraud_free.dclkv2 where year=2016 and month=8 and day=12 and request_id is not null""")
ld[k812]=readHive("""select * from etl_output.landing where year=2016 and month=8 and day=12 and request_id is not null""")
k818='818'
clk[k818]=readHive("""select * from fraud_free.dclkv2 where year=2016 and month=8 and day=18 and request_id is not null""")
ld[k818]=readHive("""select * from etl_output.landing where year=2016 and month=8 and day=18 and request_id is not null""")

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from fraud_free.dclkv2 where year=2016 and month=7 and day=12 and request_id is not null']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
, Fetched: 295028 row(s)

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from etl_output.landing where year=2016 and month=7 and day=12 and request_id is not null']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
Time taken: 2.373 seconds, Fetched: 119189 row(s)

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from fraud_free.dclkv2 where year=2016 and month=8 and day=12 and request_id is not null']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
Time taken: 3.125 seconds

In [97]:
k612='612'
clk[k612]=readHive("""select * from fraud_free.dclkv2 where year=2016 and month=6 and day=12 and request_id is not null""")
ld[k612]=readHive("""select * from etl_output.landing where year=2016 and month=6 and day=12 and request_id is not null""")

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from fraud_free.dclkv2 where year=2016 and month=6 and day=12 and request_id is not null']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
nds, Fetched: 347939 row(s)

['hive', '-e', 'set mapreduce.job.queuename=zampda_job;set hive.cli.print.header=true;select * from etl_output.landing where year=2016 and month=6 and day=12 and request_id is not null']

Logging initialized using configuration in file:/var/bh/lib/apache-hive-1.2.1-bin/conf/hive-log4j.properties
OK
conds, Fetched: 128191 row(s)



In [98]:
def hasclk(x):
    return x.query('url_y==url_y')
def hasld(x):
    return x.query('hour_y==hour_y')
def pv(x):
    return len(x['request_id'])
def uv(x):
    return x['request_id'].nunique()
def upv(x):
    return len(x['pid'])
def uuv(x):
    return x['pid'].nunique()
def stripblank(x):
    return "".join(str(x).split())
""""""
k=k612
clkld[k]=pd.merge(clk[k],ld[k],on='request_id',how='left')
clkld[k]['domain']=clkld[k]['url'].apply(f)
clkld[k]['ua']=clkld[k]['ua_x'].apply(stripblank)
clk[k]['domain']=clk[k]['url'].apply(f)
clk[k]['ua']=clk[k]['ua'].apply(stripblank)
clktotal=clk[k]
clkfraud=clk[k].query('fraud_flag==fraud_flag')
clkclean=clk[k].query('fraud_flag!=fraud_flag')
clkldtotal=clkld[k]
clkldfraud=clkld[k].query('fraud_flag==fraud_flag')
clkldclean=clkld[k].query('fraud_flag!=fraud_flag')
print 'media dist-------------------------'
print 'clk712-------------------------brief'
print pv(clktotal),uv(clktotal),uuv(clktotal),uv(hasld(clkldtotal)),uuv(hasld(clkldtotal)),'\n',\
    pv(clkfraud),uv(clkfraud),uuv(clkfraud),uv(hasld(clkldfraud)),uuv(hasld(clkldfraud)),'\n',\
    pv(clkclean),uv(clkclean),uuv(clkclean),uv(hasld(clkldclean)),uuv(hasld(clkldclean)),'\n'
temp=clkclean['vendor_id'].value_counts().reset_index()
temp['ratio']=temp['vendor_id']/pv(clkclean)
print temp
temp=clkfraud['vendor_id'].value_counts().reset_index()
temp['ratio']=temp['vendor_id']/pv(clkfraud)
print temp

media dist-------------------------
clk712-------------------------brief
347939 262442 241360 74343 69184 
85465 39177 37757 8429 8094 
262474 223855 204711 66072 61377 

    index  vendor_id     ratio
0       2      99299  0.378319
1       9      51061  0.194537
2      21      35678  0.135930
3       1      30480  0.116126
4      15      14821  0.056467
5      13       5695  0.021697
6      24       5216  0.019872
7      38       4515  0.017202
8      10       4314  0.016436
9      33       3512  0.013380
10     31       2439  0.009292
11     20       1582  0.006027
12     40       1499  0.005711
13     37       1391  0.005300
14      5        818  0.003116
15      8         98  0.000373
16     34         55  0.000210
17     56          1  0.000004
    index  vendor_id     ratio
0       2      34832  0.407559
1       9      14397  0.168455
2      10      13396  0.156743
3       1      11193  0.130966
4      21       6654  0.077856
5      15       3226  0.037746
6      37        561  0

In [100]:
print 'clk-----------domain dist'
print 'clean-----------domain dist'
# print clkclean
for v in ('1','2','9','5','15','21'):
    temp=clkclean.query('vendor_id=='+v).groupby('vendor_id')['domain'].value_counts().reset_index().head(10)
    temp['ratio']=temp[0]/len(clkclean.query('vendor_id=='+v))
    temp.index.name='no.'
    print temp
print 'fraud-----------domain dist'
for v in ('1','2','9','5','15','21'):
    temp=clkfraud.query('vendor_id=='+v).groupby('vendor_id')['domain'].value_counts().reset_index().head(10)
    temp['ratio']=temp[0]/len(clkfraud.query('vendor_id=='+v))
    temp.index.name='no.'
    print temp

clk-----------domain dist
clean-----------domain dist
     vendor_id              domain     0     ratio
no.                                               
0            1      news.ifeng.com  6813  0.223524
1            1         v.ifeng.com  5128  0.168241
2            1         www.163.com  1782  0.058465
3            1   finance.ifeng.com  1030  0.033793
4            1       ent.ifeng.com  1020  0.033465
5            1    www.lxms2004.com   894  0.029331
6            1        news.163.com   837  0.027461
7            1    www.jongbaba.com   558  0.018307
8            1  www.liuhuafang.net   521  0.017093
9            1       www.etbhw.com   509  0.016699
     vendor_id            domain     0     ratio
no.                                             
0            2        www.qq.com  7427  0.074794
1            2    aplus.pptv.com  5026  0.050615
2            2       news.qq.com  3649  0.036748
3            2       v.youku.com  3235  0.032578
4            2       www.9669.cn  2939  

In [101]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
print 'clk-----------adslot dist'
print 'clean-----------adslot dist'
for v in ('1','2','9','5','15','21'):
    temp=clkclean.query('vendor_id=='+v).groupby('vendor_id')['adslot_id'].value_counts().reset_index().head(10)
    temp['ratio']=temp[0]/len(clkclean.query('vendor_id=='+v))
    temp.index.name='no.'
    print_full(temp)
print 'fraud-----------domain dist'
for v in ('1','2','9','5','15','21'):
    temp=clkfraud.query('vendor_id=='+v).groupby('vendor_id')['adslot_id'].value_counts().reset_index().head(10)
    temp['ratio']=temp[0]/len(clkfraud.query('vendor_id=='+v))
    temp.index.name='no.'
    print_full(temp)

clk-----------adslot dist
clean-----------adslot dist
     vendor_id   adslot_id     0     ratio
no.                                       
0            1  1059014624  5128  0.168241
1            1   421343766  4476  0.146850
2            1  2792603527  1710  0.056102
3            1  3554272669  1572  0.051575
4            1  3191954303  1300  0.042651
5            1  2619718319   975  0.031988
6            1  3278159602   642  0.021063
7            1  3302429267   515  0.016896
8            1  3478903316   507  0.016634
9            1  2232019720   500  0.016404
     vendor_id                     adslot_id     0     ratio
no.                                                         
0            2   mm_26051537_2367569_9275045  4326  0.043565
1            2   mm_15191080_2147689_9068851  4023  0.040514
2            2  mm_26051537_2367569_21224084  3915  0.039426
3            2   mm_26051537_2367569_9322822  3680  0.037060
4            2  mm_26051537_2367569_21232009  3510  0.035348
5  

In [119]:
# for k,v in clk.items():
#     print v.query('adslot_id==\'1059014624\'')['url'].unique()


# print 'clk-----------ua dist'
# for v in ('1','2','9','5','15','21'):
#     print clkldclean.query('vendor_id=='+v).groupby('vendor_id')['ua'].value_counts().head(20)
#     print clkldfraud.query('vendor_id=='+v).groupby('vendor_id')['ua'].value_counts().head(20)
k=k818
clktotal=clk[k]
clkfraud=clk[k].query('fraud_flag==fraud_flag')
clkclean=clk[k].query('fraud_flag!=fraud_flag')
temp=clkclean['company_id'].value_counts().reset_index()
temp['ratio']=temp['company_id']/pv(clkclean)
print temp
temp=clkfraud['company_id'].value_counts().reset_index()
temp['ratio']=temp['company_id']/pv(clkfraud)
print temp

    index  company_id     ratio
0      20      407256  0.607721
1     104       78350  0.116916
2      12       37043  0.055277
3       9       32726  0.048835
4     270       23988  0.035796
5     116       18674  0.027866
6       8       16146  0.024094
7     568       12376  0.018468
8     630        7751  0.011566
9      13        7337  0.010949
10    352        5614  0.008377
11    190        4061  0.006060
12    482        3643  0.005436
13    601        2049  0.003058
14    342        1773  0.002646
15    371        1402  0.002092
16    629        1295  0.001932
17    507        1253  0.001870
18    645         919  0.001371
19    187         776  0.001158
20     59         755  0.001127
21    561         731  0.001091
22    613         658  0.000982
23     91         608  0.000907
24    109         483  0.000721
25    285         429  0.000640
26    636         365  0.000545
27    191         345  0.000515
28    340         332  0.000495
29    637         327  0.000488
30    46